In [65]:
%pip show esper

Name: esper
Version: 2.4.1
Summary: esper is a lightweight Entity System (ECS) for Python, with a focus on performance.
Home-page: https://github.com/benmoran56/esper
Author: Benjamin Moran
Author-email: benmoran@protonmail.com
License: MIT
Location: /Users/mton/miniconda3/envs/torch-gpu/lib/python3.9/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


#### -- [ASSET GENERATION] --
***

#### -- [EXAMPLE] --
***

In [66]:
import esper
import pygame
import math

FPS = 60
RESOLUTION = 640, 480
MOVEMENT_SPEED = 3


############################################
# Define Components
############################################
class Velocity:
    def __init__(self, vx, vy):
        self.vx = vx
        self.vy = vy


class Renderable:
    def __init__(self, posx, posy, depth=0):
        self.depth = depth
        self.x = posx
        self.y = posy


############################################
# Define Systems
############################################
class MovementProcessor(esper.Processor):
    def __init__(self, minx, maxx, miny, maxy):
        super().__init__()
        self.minx = minx
        self.maxx = maxx
        self.miny = miny
        self.maxy = maxy

    def process(self):
        # Process all entities with both Velocity and Renderable
        for ent, (vel, rend) in self.world.get_components(Velocity, Renderable):
            # Update renderable position by velocity
            rend.x += vel.vx  # type: ignore
            rend.y += vel.vy  # type: ignore


class RenderProcessor(esper.Processor):
    def __init__(self, window, clear_color=(1, 0, 0)):
        super().__init__()
        self.window = window
        self.clear_color = clear_color

    def process(self):
        # Clear screen
        self.window.fill(self.clear_color)
        # Blit all entities with Renderable
        for ent, rend in self.world.get_component(Renderable):
            pygame.draw.rect(self.window, (255, 255, 0), (rend.x, rend.y, 64, 64), 0)
        pygame.display.flip()


############################################
# Event Handle
############################################
def handle_events(velocity: Velocity) -> bool:
    frame_events = pygame.event.get()
    if not frame_events:
        return True
    for event in frame_events:
        if event.type == pygame.QUIT:
            return False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                velocity.vy = -MOVEMENT_SPEED
            if event.key == pygame.K_DOWN:
                velocity.vy = MOVEMENT_SPEED
            if event.key == pygame.K_RIGHT:
                velocity.vx = MOVEMENT_SPEED
            if event.key == pygame.K_LEFT:
                velocity.vx = -MOVEMENT_SPEED
        elif event.type == pygame.KEYUP:
            if event.key in (pygame.K_UP, pygame.K_DOWN):
                velocity.vy = 0
            if event.key in (pygame.K_LEFT, pygame.K_RIGHT):
                velocity.vx = 0
    # normalize velocity vector
    length = math.sqrt(velocity.vx**2 + velocity.vy**2)
    if length > 0:
        velocity.vx /= length
        velocity.vy /= length
        velocity.vx *= MOVEMENT_SPEED
        velocity.vy *= MOVEMENT_SPEED
    return True


############################################
# Main Loop
############################################
def run():
    # Init pygame
    pygame.init()
    window = pygame.display.set_mode(RESOLUTION)
    clock = pygame.time.Clock()
    pygame.key.set_repeat(1, 1)
    # Inite ECS
    world = esper.World()
    player = world.create_entity()
    world.add_component(player, Velocity(vx=0, vy=0))
    world.add_component(player, Renderable(posx=100, posy=100))
    # Init Systems
    render_processor = RenderProcessor(window=window)
    movement_processor = MovementProcessor(
        minx=0, maxx=RESOLUTION[0], miny=0, maxy=RESOLUTION[1]
    )
    world.add_processor(render_processor)
    world.add_processor(movement_processor)

    running = True
    while running:
        # Handle events
        running = handle_events(world.component_for_entity(player, Velocity))
        # Update ECS
        world.process()
        # Clock tick
        clock.tick(FPS)


if __name__ == "__main__":
    run()
    pygame.quit()


#### --- [FOOTNOTES] --
***

Roadmap for learning ECS in python
***

ECS (Entity Component System) is a popular architectural pattern used in game 
development and other real-time applications. It's designed to separate data and 
functionality, which makes it easy to manage the complexity of systems with many 
interacting entities. Here's a roadmap to learn ECS in Python:

1. **Understand the basics of ECS**

   - Read articles and watch videos about ECS fundamentals:
     - [Understanding Entity Component System](https://www.gamedev.net/tutorials/programming/general-and-gameplay-programming/understanding-component-entity-systems-r3013/)
     - [What is an Entity Component System?](https://www.youtube.com/watch?v=NTWSeQtHZ9M)

2. **Learn Python**

   - If you're new to Python, follow a tutorial or course to learn the basics:
     - [Python.org Beginner's Guide](https://docs.python.org/3/tutorial/index.html)
     - [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

3. **Study existing ECS libraries in Python**

   - Look at popular Python ECS libraries to understand how they implement the 
   pattern:
     - [esper](https://github.com/benmoran56/esper)
     - [ecs](https://github.com/Samathy/ecspy)

4. **Create a simple game or application using ECS in Python**

   - Start with a simple project, like a basic 2D game or simulation, to apply 
   your ECS knowledge:
     - Create the main game loop
     - Define entities, components, and systems
     - Implement a basic rendering system
     - Add user input handling
     - Implement game logic, such as movement and collisions

5. **Expand your knowledge with advanced topics**

   - Study more advanced ECS concepts and optimization techniques:
     - Data-oriented design
     - Multithreading and parallelism
     - Spatial partitioning and efficient queries

6. **Contribute to open-source projects or create your own ECS library**

   - Once you feel comfortable with ECS, consider contributing to existing 
   projects or creating your own library to share with the community.

7. **Network and join game development communities**

   - Participate in game development forums, meetups, and conferences to learn 
   from others, share your knowledge, and stay up-to-date with the latest ECS trends and best practices.

By following this roadmap, you'll get a solid understanding of ECS in Python and 
be ready to create efficient and maintainable real-time applications.

Explain the difference between pygame.display.flip() and pygame.display.update()
***

Both `pygame.display.flip()` and `pygame.display.update()` are used to update  
the display surface in Pygame, but they have slightly different behaviors:

1. `pygame.display.flip()`: This function updates the entire screen, swapping  
the front and back display buffers. This is called double-buffering, where you  
draw to an off-screen buffer, and once the drawing is complete, you flip the  
buffers to display the new content. It is more suitable for games or  
applications that require frequent updates to the entire screen.

   Example usage:

   ```python
   pygame.display.flip()
   ```

2. `pygame.display.update()`: This function updates only the specified  
portion(s) of the screen. If you provide no arguments or pass an empty list, it  
updates the entire screen, similar to `pygame.display.flip()`. However, if you  
pass a rectangle or a list of rectangles as an argument, only those areas of  
the screen will be updated. This can be more efficient when you only need to  
update specific parts of the screen, as it requires less processing.

   Example usage:

   ```python
   pygame.display.update()  # Updates the entire screen
   pygame.display.update(rect)  # Updates a single rectangle
   pygame.display.update([rect1, rect2, rect3])  # Updates multiple rectangles
   ```

In summary, use `pygame.display.flip()` when you need to update the entire  
screen, usually in applications that require constant screen redraws. Use  
`pygame.display.update()` when you want to update only specific portions of the  
screen, which can provide performance benefits if only small areas need to be  
redrawn.